In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('../data/cleaned_rounds_data_with_stats.csv')
df.head()

,round_winner,team_1_position,team_1_win_percentage,team_2_position,team_2_win_percentage,player_1_team_name,player_1_name,player_1_rating,player_1_kills_per_round,player_1_headshots,...,player_10_CZ75-Auto,player_10_Desert Eagle,player_10_R8 Revolver,player_10_Dual Berettas,player_10_Smoke Grenade,player_10_High Explosive Grenade,player_10_Flashbang,player_10_Incendiary Grenade,player_10_Molotov,player_10_Decoy Grenade
0,1,30,58.1,5,66.7,G2 Esports,NiKo,1.18,0.72,51.8,...,0,0,0,0,1,0,1,1,0,0
1,1,30,58.1,5,66.7,G2 Esports,NiKo,1.18,0.72,51.8,...,0,0,0,0,1,1,1,1,0,0
2,0,30,58.1,5,66.7,G2 Esports,NiKo,1.18,0.72,51.8,...,0,0,0,0,1,1,0,1,0,0
3,1,30,58.1,5,66.7,G2 Esports,NiKo,1.18,0.72,51.8,...,0,0,0,0,1,1,0,1,0,0
4,1,30,58.1,5,66.7,G2 Esports,NiKo,1.18,0.72,51.8,...,0,0,0,0,1,1,1,1,0,0


In [5]:
df = df.drop(columns=[f'player_{i}_{suffix}' for i in range(1, 11) for suffix in ['team_name', 'name']])
df.head()

,round_winner,team_1_position,team_1_win_percentage,team_2_position,team_2_win_percentage,player_1_rating,player_1_kills_per_round,player_1_headshots,player_1_deaths_per_round,player_1_rounds_contributed,...,player_10_CZ75-Auto,player_10_Desert Eagle,player_10_R8 Revolver,player_10_Dual Berettas,player_10_Smoke Grenade,player_10_High Explosive Grenade,player_10_Flashbang,player_10_Incendiary Grenade,player_10_Molotov,player_10_Decoy Grenade
0,1,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,...,0,0,0,0,1,0,1,1,0,0
1,1,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,...,0,0,0,0,1,1,1,1,0,0
2,0,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,...,0,0,0,0,1,1,0,1,0,0
3,1,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,...,0,0,0,0,1,1,0,1,0,0
4,1,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,...,0,0,0,0,1,1,1,1,0,0


In [6]:
X = df.drop(['round_winner'], axis=1)
X.head()

,team_1_position,team_1_win_percentage,team_2_position,team_2_win_percentage,player_1_rating,player_1_kills_per_round,player_1_headshots,player_1_deaths_per_round,player_1_rounds_contributed,player_1_team_num,...,player_10_CZ75-Auto,player_10_Desert Eagle,player_10_R8 Revolver,player_10_Dual Berettas,player_10_Smoke Grenade,player_10_High Explosive Grenade,player_10_Flashbang,player_10_Incendiary Grenade,player_10_Molotov,player_10_Decoy Grenade
0,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,0,...,0,0,0,0,1,0,1,1,0,0
1,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,0,...,0,0,0,0,1,1,1,1,0,0
2,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,0,...,0,0,0,0,1,1,0,1,0,0
3,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,0,...,0,0,0,0,1,1,0,1,0,0
4,30,58.1,5,66.7,1.18,0.72,51.8,0.64,73.5,0,...,0,0,0,0,1,1,1,1,0,0


In [7]:
y = df['round_winner']

In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [14]:
#round 1
param_grid = {
   'max_depth': [3, 4, 5],
   'learning_rate': [0.1, 0.01, 0.05],
   'gamma': [0, 0.25, 1.0],
   'reg_lambda': [0.1, 1.0, 10.0],
   'scale_pos_weight': [1, 3, 5]
}

first round optimal parameters
{'gamma': 0, 'learning_rate': 0.05, 'max_depth': 5, 'reg_lambda': 0.1, 'scale_pos_weight': 1}

In [17]:
#round 2
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.03, 0.01],
    'gamma': [0.0],
    'reg_lambda': [0.1, 0.5 ,1.0],
    'scale_pos_weight': [1]
}

second round optimal parameters 
{'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 7, 'reg_lambda': 0.1, 'scale_pos_weight': 1}

In [20]:
#round 3
param_grid = {
    'max_depth': [5, 7, 9],
    'learning_rate': [0.05],
    'gamma': [0.0],
    'reg_lambda': [0.1, 0.2, 0.3],
    'scale_pos_weight': [1]
}

third round optimal parameters 
{'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 9, 'reg_lambda': 0.1, 'scale_pos_weight': 1}

In [23]:
#round 4
param_grid = {
    'max_depth': [9, 11, 13],
    'learning_rate': [0.05],
    'gamma': [0],
    'reg_lambda': [0.1],
    'scale_pos_weight': [1]
}

forth round optimal parameters {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 9, 'reg_lambda': 0.1, 'scale_pos_weight': 1}

In [24]:
optimal_params = GridSearchCV(
   estimator=XGBClassifier(objective='binary:logistic', seed=42, eval_metric='aucpr',use_label_encoder=False),
   param_grid=param_grid,
   scoring='roc_auc',
   verbose=0,
   cv=3
)

In [25]:
optimal_params.fit(X_train,
           y_train)

print(optimal_params.best_params_)

{'gamma': 0, 'learning_rate': 0.05, 'max_depth': 9, 'reg_lambda': 0.1, 'scale_pos_weight': 1}
